In [27]:
# %%python
import os
import re
from pathlib import Path
import pandas as pd
import numpy as np

# sklearn voor metrics
from sklearn.metrics import accuracy_score, f1_score

# ===== Helpers =====

POS_ALIASES = {"positive", "positief", "pos", "pos."}
NEG_ALIASES = {"negative", "negatief", "neg", "neg."}
MIXED_ALIASES = {"mixed", "gemengd", "neutraal", "neutral", "mix", "both"}

def norm_label(x: str | float | None) -> str | None:
    """Normaliseer losse labels naar 'positive'/'negative' (of None)."""
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return None
    s = str(x).strip().lower()
    # verwijder quotes/punctuatie
    s = s.strip('“”"\' .:;!?')
    if s in POS_ALIASES:
        return "positive"
    if s in NEG_ALIASES:
        return "negative"
    if s in MIXED_ALIASES or s in {"", "none", "nan"}:
        return None
    # soms modellen geven + / - of woorden met extra context
    if s in {"+", "plus"}:
        return "positive"
    if s in {"-", "min", "minus"}:
        return "negative"
    # heuristiek: als het antwoord alleen het woord 'positief/positive' of 'negatief/negative' bevat aan begin
    if s.startswith("posit"):
        return "positive"
    if s.startswith("negat"):
        return "negative"
    return None

def majority_from_parts(row: pd.Series, base: str) -> str | None:
    """Bouw overall label uit *_title_label, *_lead_label, *_body_label als *_overall_label ontbreekt."""
    parts = []
    for p in ("title", "lead", "body"):
        col = f"{base}_{p}_label"
        if col in row and pd.notna(row[col]):
            parts.append(norm_label(row[col]))
    parts = [p for p in parts if p in {"positive", "negative"}]
    if not parts:
        return None
    # majority vote
    pos = parts.count("positive")
    neg = parts.count("negative")
    if pos > neg:
        return "positive"
    if neg > pos:
        return "negative"
    # tie-breaker: kies negative conservatief
    return "negative"

def detect_id_column(df: pd.DataFrame) -> str:
    """Vind 'id' kolom (case-insensitive) of raise met duidelijke boodschap."""
    for c in df.columns:
        if str(c).strip().lower() == "id":
            return c
    # zoek eventueel 'article_id' of 'doc_id'
    for cand in df.columns:
        if str(cand).strip().lower() in {"article_id","doc_id","document_id,", "artikel"}:
            return cand
    raise ValueError(f"Geen 'id' kolom gevonden. Beschikbare kolommen: {list(df.columns)}")

def pick_human_label_column(df: pd.DataFrame) -> str:
    """
    Kies de human/true label kolom. Heuristiek: kolomnamen met 'human' of 'true' of 'label'
    waarvan de waarden bekende sentimenten bevatten (incl. Mixed).
    """
    candidates = []
    for c in df.columns:
        lc = str(c).lower()
        if any(k in lc for k in ("human","true","label","sentiment","gold","reference")):
            vals = set(str(x).strip().lower() for x in df[c].dropna().unique().tolist()[:100])
            if any(v in POS_ALIASES | NEG_ALIASES | MIXED_ALIASES for v in vals):
                candidates.append(c)
    if not candidates:
        # fallback: probeer 'sentiment' expliciet
        for c in df.columns:
            if str(c).lower() == "sentiment":
                return c
        raise ValueError("Kon geen human/true label kolom vinden. Controleer kolomnamen.")
    # geef voorkeur aan naam met 'human' of 'true'
    for c in candidates:
        lc = str(c).lower()
        if "human" in lc or "true" in lc:
            return c
    return candidates[0]


In [28]:
# %%python
DATA_DIR = Path("out")  # pas aan als je bestanden elders staan

# Bestanden (consistent met wat je aangaf)
PATH_HUMAN      = Path("out/Human_Sentiment.xlsx")
PATH_LLM        = Path("out/newspapers_sentiment_llms.xlsx")
PATH_NLI        = Path("out/nli_results.csv")          # NB: CSV volgens jouw aanlevering
PATH_SENTIMENT  = Path("out/sentiment_results.csv")    # NB: CSV

# Inlezen
df_human = pd.read_excel(PATH_HUMAN)
df_llms  = pd.read_excel(PATH_LLM)
df_nli   = pd.read_csv(PATH_NLI)
df_sent  = pd.read_csv(PATH_SENTIMENT)

# ID & label kolommen
id_col_human = detect_id_column(df_human)
lab_col_human = pick_human_label_column(df_human)

# Normaliseer human labels en filter Mixed/None eruit
df_human["_human_label"] = df_human[lab_col_human].map(norm_label)
mask_valid = df_human["_human_label"].isin({"positive","negative"})
df_human_valid = df_human.loc[mask_valid, [id_col_human, "_human_label"]].rename(
    columns={id_col_human: "id"}
).reset_index(drop=True)

print(f"Human records (valid, excl. Mixed): {len(df_human_valid)}")
df_human_valid.head()

ALL_IDS = df_human["Artikel"].dropna().astype(str).tolist()
N_ALL = len(ALL_IDS)
print("Totaal aantal artikelen (incl. Mixed):", N_ALL)


Human records (valid, excl. Mixed): 65
Totaal aantal artikelen (incl. Mixed): 70


In [29]:
# %%python
def extract_models_from_llm(df_llms: pd.DataFrame) -> dict[str, pd.DataFrame]:
    """
    Zoek alle *_overall_label kolommen; als die ontbreken maar *_title/lead/body_label bestaan,
    maak dan een overall kolom via majority. Retourneer dict: {model_name: df[['id','pred']]}
    """
    id_col = detect_id_column(df_llms)
    models = {}

    # 3a. vind alle 'bases' op basis van *_overall_label
    overall_cols = [c for c in df_llms.columns if c.endswith("_overall_label")]
    bases_from_overall = [c.replace("_overall_label","") for c in overall_cols]

    # 3b. vind extra bases die alleen *_title/lead/body_label hebben
    part_label_cols = [c for c in df_llms.columns if c.endswith(("_title_label","_lead_label","_body_label"))]
    bases_from_parts = sorted({re.sub(r"_(title|lead|body)_label$","",c) for c in part_label_cols})

    bases = sorted(set(bases_from_overall) | set(bases_from_parts))

    for base in bases:
        out = df_llms[[id_col]].copy().rename(columns={id_col:"id"})
        col_overall = f"{base}_overall_label"
        if col_overall in df_llms.columns:
            out["pred"] = df_llms[col_overall].map(norm_label)
        else:
            # majority uit parts
            tmp = df_llms.copy()
            out["pred"] = tmp.apply(lambda r: majority_from_parts(r, base), axis=1)
        # filter None
        out = out[out["pred"].isin({"positive","negative"})]
        if not out.empty:
            # mooi naammetje
            model_name = base
            models[model_name] = out[["id","pred"]].copy()
    return models

def extract_models_from_generic(df_any: pd.DataFrame) -> dict[str, pd.DataFrame]:
    """
    Zelfde idee voor NLI/sentiment CSVs. Probeert eerst *_overall_label,
    anders majority uit *_title/lead/body_label, en als dat er niet is,
    zoekt naar kolommen die eindigen op '_label' (maar niet title/lead/body).
    """
    id_col = detect_id_column(df_any)
    models = {}

    overall_cols = [c for c in df_any.columns if c.endswith("_overall_label")]
    bases_from_overall = [c[:-len("_overall_label")] for c in overall_cols]

    part_label_cols = [c for c in df_any.columns if c.endswith(("_title_label","_lead_label","_body_label"))]
    bases_from_parts = sorted({re.sub(r"_(title|lead|body)_label$","",c) for c in part_label_cols})

    # fallback: *_label kolommen die geen title/lead/body en geen overall zijn
    plain_label_cols = [
        c for c in df_any.columns
        if c.endswith("_label")
        and not c.endswith(("_title_label","_lead_label","_body_label","_overall_label"))
    ]
    bases_from_plain = [c[:-len("_label")] for c in plain_label_cols]

    bases = sorted(set(bases_from_overall) | set(bases_from_parts) | set(bases_from_plain))

    for base in bases:
        out = df_any[[id_col]].copy().rename(columns={id_col:"id"})
        col_overall = f"{base}_overall_label"
        if col_overall in df_any.columns:
            out["pred"] = df_any[col_overall].map(norm_label)
        else:
            # probeer parts
            has_parts = any(f"{base}_{p}_label" in df_any.columns for p in ("title","lead","body"))
            if has_parts:
                out["pred"] = df_any.apply(lambda r: majority_from_parts(r, base), axis=1)
            else:
                # probeer plain *_label
                col_plain = f"{base}_label"
                if col_plain in df_any.columns:
                    out["pred"] = df_any[col_plain].map(norm_label)
                else:
                    continue
        out = out[out["pred"].isin({"positive","negative"})]
        if not out.empty:
            models[base] = out[["id","pred"]].copy()
    return models


In [30]:
# %%python
def evaluate_against_human(df_human_valid: pd.DataFrame, preds: dict[str, pd.DataFrame]) -> pd.DataFrame:
    """
    preds: dict {model_name: DataFrame[id, pred]}
    Retourneer tabel met accuracy en F1 per model (alleen op intersectie van id's).
    """
    rows = []
    true_map = df_human_valid.set_index("id")["_human_label"]
    for model_name, df_pred in preds.items():
        dfp = df_pred.copy()
        dfp = dfp[dfp["id"].isin(true_map.index)]
        if dfp.empty:
            continue
        y_true = true_map.loc[dfp["id"]].values
        y_pred = dfp["pred"].values
        # filter mogelijke None (zou niet moeten voorkomen door eerdere filter)
        mask = pd.Series(y_pred).isin({"positive","negative"}).values & pd.Series(y_true).isin({"positive","negative"}).values
        y_true = np.array(y_true)[mask]
        y_pred = np.array(y_pred)[mask]
        if y_true.size == 0:
            continue
        acc = accuracy_score(y_true, y_pred)
        f1  = f1_score(y_true, y_pred, pos_label="positive")
        rows.append({"model": model_name, "n": int(y_true.size), "accuracy": acc, "f1": f1})
    if not rows:
        return pd.DataFrame(columns=["model","n","accuracy","f1"])
    out = pd.DataFrame(rows).sort_values(["f1","accuracy","n"], ascending=[False, False, False]).reset_index(drop=True)
    return out


In [31]:
# %%python
# Extractie per bron
preds_llm = extract_models_from_llm(df_llms)
preds_nli = extract_models_from_generic(df_nli)
preds_sent= extract_models_from_generic(df_sent)

# Combineer alle modellen
all_preds = {}
for d in (preds_llm, preds_nli, preds_sent):
    for k, v in d.items():
        # als dezelfde key al bestaat (zelfde base), kies de grootste dekking (meer n)
        if k in all_preds:
            n_old = len(all_preds[k])
            n_new = len(v)
            if n_new > n_old:
                all_preds[k] = v
        else:
            all_preds[k] = v

print(f"Aantal modellen gevonden: {len(all_preds)}")
list(all_preds.keys())[:10]


Aantal modellen gevonden: 8


['GPT_4_0',
 'Groq_Llama_70B',
 'HF_Llama_2_13B_dutch',
 'bert_base_dutch_cased_finetuned_snli',
 'mDeBERTa_v3_base_mnli_xnli',
 'robbert_v2_dutch_finetuned_snli',
 'robbertje_dutch_finetuned_snli',
 'rob']

In [32]:
# %%python
summary = evaluate_against_human(df_human_valid, all_preds)
summary

# Opslaan
OUT_DIR = Path("out")
OUT_DIR.mkdir(parents=True, exist_ok=True)
summary.to_excel(OUT_DIR / "eval_summary.xlsx", index=False)
summary.to_csv(OUT_DIR / "eval_summary.csv", index=False)
print("Geschreven naar:", OUT_DIR / "eval_summary.xlsx", "en", OUT_DIR / "eval_summary.csv")


Geschreven naar: out\eval_summary.xlsx en out\eval_summary.csv


In [33]:
# %%python
# Check paar bekende modellen als ze bestaan
for key in ["rob", "mDeBERTa_v3_base_mnli_xnli", "Groq_Llama_70B", "GPT_4_0", "robbert_v2_dutch_finetuned_snli"]:
    k = key if key in all_preds else None
    if k:
        joined = df_human_valid.merge(all_preds[k], on="id", how="inner", suffixes=("_true","_pred"))
        print(f"\n{k}: n={len(joined)}  acc={accuracy_score(joined._human_label, joined.pred):.3f}  f1={f1_score(joined._human_label, joined.pred, pos_label='positive'):.3f}")



rob: n=65  acc=0.738  f1=0.805

mDeBERTa_v3_base_mnli_xnli: n=65  acc=0.800  f1=0.843

Groq_Llama_70B: n=65  acc=0.862  f1=0.877

GPT_4_0: n=65  acc=0.815  f1=0.824

robbert_v2_dutch_finetuned_snli: n=65  acc=0.662  f1=0.776


In [34]:
# %%python
import pandas as pd
import numpy as np

def percent_positive_all(all_ids: list[str], df_pred: pd.DataFrame) -> dict:
    """
    all_ids: volledige lijst met id's (incl. Mixed)
    df_pred: DataFrame met kolommen ['id','pred'] (al genormaliseerd door extractie)
    Retourneert dict met: {'n_total': N, 'n_positive': k, 'positive_pct_all': 100*k/N}
    Ontbrekende voorspellingen tellen als 'niet-positief' (dus k verandert niet, N blijft 70).
    """
    # maak mapping id -> genormaliseerd label
    pred_map = df_pred.dropna(subset=["id"]).copy()
    pred_map["id"] = pred_map["id"].astype(str)
    pred_map = pred_map.set_index("id")["pred"]

    pos = 0
    for _id in all_ids:
        lab = pred_map.get(str(_id), None)
        if lab == "positive":  # alleen 'positive' telt
            pos += 1

    n_total = len(all_ids)
    pct = (pos / n_total) * 100.0 if n_total else np.nan
    return {"n_total": n_total, "n_positive": pos, "positive_pct_all": pct}

# 1) Maak tabel met % positief per model
rows_pos = []
for model_name, df_pred in all_preds.items():
    stats = percent_positive_all(ALL_IDS, df_pred)
    rows_pos.append({"model": model_name, **stats})
df_pos = pd.DataFrame(rows_pos).sort_values("positive_pct_all", ascending=False)

# 2) Combineer met je bestaande evaluatie (accuracy/F1 op subset zonder Mixed)
#    'summary' komt uit je eerdere cel evaluate_against_human(...)
summary_full = df_pos.merge(summary, on="model", how="left")

# 3) Mooie kolomvolgorde + export
cols = ["model", "n_total", "n_positive", "positive_pct_all", "n", "accuracy", "f1"]
summary_full = summary_full.reindex(columns=cols)

# 4) Schrijf weg (overschrijf eerdere evaluatiebestanden)
OUT_DIR = Path("out")
OUT_DIR.mkdir(parents=True, exist_ok=True)
summary_full.to_excel(OUT_DIR / "eval_summary.xlsx", index=False)
summary_full.to_csv(OUT_DIR / "eval_summary.csv", index=False)

print("Geüpdatet:", OUT_DIR / "eval_summary.xlsx")
display(summary_full.head(10))


Geüpdatet: out\eval_summary.xlsx


,model,n_total,n_positive,positive_pct_all,n,accuracy,f1
0,HF_Llama_2_13B_dutch,70,70,100.000000,65,0.615385,0.761905
1,bert_base_dutch_cased_finetuned_snli,70,69,98.571429,65,0.630769,0.769231
2,robbert_v2_dutch_finetuned_snli,70,63,90.000000,65,0.661538,0.775510
3,rob,70,51,72.857143,65,0.738462,0.804598
4,mDeBERTa_v3_base_mnli_xnli,70,46,65.714286,65,0.800000,0.843373
5,Groq_Llama_70B,70,36,51.428571,65,0.861538,0.876712
6,GPT_4_0,70,30,42.857143,65,0.815385,0.823529
7,robbertje_dutch_finetuned_snli,70,28,40.000000,65,0.553846,0.553846


# Human Analyse per Krant etc

In [35]:
import pandas as pd
from pathlib import Path

# Pad naar bestanden
PATH_HUMAN = Path("out/Human_Sentiment.xlsx")
PATH_TLB   = Path("out/Title_Lead_Body.xlsx")

# 1) Inlezen
df_human = pd.read_excel(PATH_HUMAN)
df_tlb   = pd.read_excel(PATH_TLB)

# Kolomnamen opschonen
df_human = df_human.loc[:, ~df_human.columns.str.startswith("Unnamed:")]
df_tlb   = df_tlb.loc[:,   ~df_tlb.columns.str.startswith("Unnamed:")]

# Zorg dat id kolommen consistent zijn
df_human.rename(columns={"Artikel": "id"}, inplace=True)
df_tlb.rename(columns={"Artikel": "id"}, inplace=True)

# 2) Merge human sentiment met krant-info
df_merged = pd.merge(df_human, df_tlb[["id","bron"]], on="id", how="left")

# 3) Normaliseer sentiment labels
def norm_label_raw(x):
    if pd.isna(x):
        return None
    x = str(x).strip().lower()
    if x in ["positive", "positief", "+"]:
        return "positive"
    elif x in ["negative", "negatief", "-"]:
        return "negative"
    elif x in ["mixed"]:
        return "mixed"
    return None

df_merged["_sentiment"] = df_merged["Sentiment"].map(norm_label_raw)

# 4) Bereken percentage positief totaal
n_total = len(df_merged)
n_pos   = (df_merged["_sentiment"] == "positive").sum()
pct_pos_total = (n_pos / n_total) * 100 if n_total > 0 else None

print(f"Totaal aantal artikelen: {n_total}")
print(f"Aantal 'positive': {n_pos} → {pct_pos_total:.2f}% positief (inclusief Mixed en Negative)")

# 5) Bereken percentage positief PER krant
summary_per_krant = (
    df_merged.groupby("bron")
    .apply(lambda g: pd.Series({
        "n_total": len(g),
        "n_positive": (g["_sentiment"] == "positive").sum(),
        "positive_pct": 100 * (g["_sentiment"] == "positive").sum() / len(g) if len(g) > 0 else None
    }))
    .reset_index()
    .sort_values("positive_pct", ascending=False)
)

# 6) Output
print("\nPercentage positief per krant:")
display(summary_per_krant)

# Opslaan naar Excel
OUT_PATH = Path("out/human_sentiment_by_newspaper.xlsx")
summary_per_krant.to_excel(OUT_PATH, index=False)
print(f"\nResultaat opgeslagen naar {OUT_PATH}")


Totaal aantal artikelen: 70
Aantal 'positive': 40 → 57.14% positief (inclusief Mixed en Negative)

Percentage positief per krant:


C:\Users\semde\AppData\Local\Temp\ipykernel_20804\2568840311.py:49: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


,bron,n_total,n_positive,positive_pct
1,AD De Dordtenaar,1.0,1.0,100.000000
2,AD Groene Hart,1.0,1.0,100.000000
3,Dagblad van het Noorden,3.0,3.0,100.000000
10,de Stentor,2.0,2.0,100.000000
7,Noordhollands Dagblad,8.0,7.0,87.500000
5,De Telegraaf,7.0,6.0,85.714286
9,de Gelderlander,6.0,5.0,83.333333
8,Trouw,8.0,4.0,50.000000
4,De Limburger,2.0,1.0,50.000000
6,NRC,14.0,5.0,35.714286



Resultaat opgeslagen naar out\human_sentiment_by_newspaper.xlsx
